In [ ]:
from dataclasses import dataclass
import matplotlib.pyplot as plt

# Function to calculate Basal Metabolic Rate (BMR) using the Harris-Benedict equation
def calculate_bmr(gender, weight, height, age):
    if gender.lower() == 'male':
        bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    elif gender.lower() == 'female':
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)
    else:
        raise ValueError("Gender must be 'male' or 'female'")
    return bmr

# Function to calculate Total Daily Energy Expenditure (TDEE) based on BMR and activity level
def calculate_tdee(bmr, activity_level):
    activity_factors = {
        "sedentary": 1.2,
        "lightly active": 1.375,
        "moderately active": 1.55,
        "very active": 1.725,
        "super active": 1.9
    }
    
    if activity_level not in activity_factors:
        raise ValueError("Invalid activity level. Choose from: 'sedentary', 'lightly active', 'moderately active', 'very active', 'super active'")
    
    tdee = bmr * activity_factors[activity_level]
    return tdee

# Function to calculate fat goal based on percentage of TDEE
def calculate_fat_goal(tdee, fat_percentage):
    calories_from_fat = tdee * (fat_percentage / 100.0)
    grams_of_fat = round(calories_from_fat / 9)  # 1 gram of fat = 9 calories
    return grams_of_fat

# Function to calculate carbs goal based on percentage of TDEE
def calculate_carbs_goal(tdee, carbs_percentage):
    calories_from_carbs = tdee * (carbs_percentage / 100.0)
    grams_of_carbs = round(calories_from_carbs / 4)  # 1 gram of carbs = 4 calories
    return grams_of_carbs

# Main function to calculate calorie goals based on user input
def calculate_calorie_goals(gender, weight, height, age, activity_level, goal):
    global CALORIE_GOAL_LIMIT, FAT_GOAL, CARBS_GOAL
    
    bmr = calculate_bmr(gender, weight, height, age)
    tdee = calculate_tdee(bmr, activity_level)
    tdee_int = round(tdee)
    
    if goal == 'maintain':
        CALORIE_GOAL_LIMIT = tdee_int
        result = f"TDEE (to maintain current weight): {tdee_int} calories/day"
    elif goal == 'lose':
        calorie_goal_loss_min = round(tdee - 500)
        calorie_goal_loss_max = round(tdee - 1000)
        CALORIE_GOAL_LIMIT = calorie_goal_loss_min
        result = f"Calorie Goal for Weight Loss: {calorie_goal_loss_min} to {calorie_goal_loss_max} calories/day"
    elif goal == 'bulk':
        calorie_goal_bulk_min = round(tdee + 250)
        calorie_goal_bulk_max = round(tdee + 500)
        CALORIE_GOAL_LIMIT = calorie_goal_bulk_min
        result = f"Calorie Goal for Bulking: {calorie_goal_bulk_min} to {calorie_goal_bulk_max} calories/day"
    else:
        raise ValueError("Invalid goal. Choose from: 'maintain', 'lose', 'bulk'")
    
    print(f"\nResults:\n{result}")
    print(f"CALORIE_GOAL_LIMIT set to: {CALORIE_GOAL_LIMIT} calories/day")

    # Calculate and display protein goal (assumed to be 30% of TDEE)
    PROTEIN_GOAL = int(weight * 2)  # Set PROTEIN_GOAL to 2 times the weight
    print(f"PROTEIN_GOAL set to: {PROTEIN_GOAL} grams/day")

    # Calculate and display fat goal
    FAT_GOAL = calculate_fat_goal(CALORIE_GOAL_LIMIT, fat_percentage=30)  # Assuming 30% of calories from fat
    print(f"FAT_GOAL set to: {FAT_GOAL} grams/day")
    
    # Calculate and display carbs goal
    CARBS_GOAL = calculate_carbs_goal(CALORIE_GOAL_LIMIT, carbs_percentage=50)  # Assuming 50% of calories from carbs
    print(f"CARBS_GOAL set to: {CARBS_GOAL} grams/day")

# Function to get user input and validate it
def get_user_input():
    global PROTEIN_GOAL
    
    gender = input("Enter your gender (male/female): ").strip().lower()
    if gender not in ['male', 'female']:
        raise ValueError("Invalid input for gender. Please enter 'male' or 'female'.")

    weight = float(input("Enter your weight (in kg): ").strip())
    global PROTEIN_GOAL
    PROTEIN_GOAL = int(weight * 2)  # Set PROTEIN_GOAL to 2 times the weight
    
    height = float(input("Enter your height (in cm): ").strip())
    age = int(input("Enter your age (in years): ").strip())
    
    print("\nChoose your activity level from the options below:")
    print("1. Sedentary (little or no exercise)")
    print("2. Lightly active (light exercise/sports 1-3 days/week)")
    print("3. Moderately active (moderate exercise/sports 3-5 days/week)")
    print("4. Very active (hard exercise/sports 6-7 days a week)")
    print("5. Super active (very hard exercise, physical job, or training twice a day)")
    
    activity_level_choice = input("Enter the number corresponding to your activity level (1-5): ").strip()

    activity_levels = {
        '1': 'sedentary',
        '2': 'lightly active',
        '3': 'moderately active',
        '4': 'very active',
        '5': 'super active'
    }
    
    if activity_level_choice not in activity_levels:
        raise ValueError("Invalid input for activity level. Please enter a number between 1 and 5.")
    
    activity_level = activity_levels[activity_level_choice]
    
    print("\nChoose your goal:")
    print("1. Maintain weight")
    print("2. Lose weight")
    print("3. Bulk (gain weight)")
    
    goal_choice = input("Enter the number corresponding to your goal (1-3): ").strip()

    goals = {
        '1': 'maintain',
        '2': 'lose',
        '3': 'bulk'
    }
    
    if goal_choice not in goals:
        raise ValueError("Invalid input for goal. Please enter a number between 1 and 3.")
    
    goal = goals[goal_choice]
    
    return gender, weight, height, age, activity_level, goal

# Define CALORIE_GOAL_LIMIT and other nutrient goals
CALORIE_GOAL_LIMIT = 0  # Calories
PROTEIN_GOAL = 0        # grams
FAT_GOAL = 0            # grams
CARBS_GOAL = 0          # grams

# List to keep track of food entries for today
today = []

@dataclass
class Food:
    name: str
    calories: int
    protein: int
    fat: int
    carbs: int

# Start the main program
print("Hello! Welcome to the Calorie and Nutrient Tracker.")

# Step 1: Get TDEE and nutritional goals
gender, weight, height, age, activity_level, goal = get_user_input()
calculate_calorie_goals(gender, weight, height, age, activity_level, goal)

done = False
while not done:
    print("""
    (1) Add a new food
    (2) Visualize Progress
    (3) Quit
    """)

    choice = input("Choose an option: ")
    
    if choice == "1":
        print("Adding a new food!")
        name = input("Name: ")
        calories = int(input("Calories: "))
        protein = int(input("Protein: "))
        fat = int(input("Fat: "))
        carbs = int(input("Carbs: "))
        food = Food(name, calories, protein, fat, carbs)
        today.append(food)  # Append food to the today list
        print("Successfully added!")
        
    elif choice == "2":
        calorie_sum = sum(food.calories for food in today)
        protein_sum = sum(food.protein for food in today)
        fat_sum = sum(food.fat for food in today)
        carbs_sum = sum(food.carbs for food in today)

        fig, axs = plt.subplots(2, 2)
        
        # Pie chart for macronutrient distribution
        axs[0, 0].pie([protein_sum, fat_sum, carbs_sum], labels=["Proteins", "Fats", "Carbs"], autopct="%1.1f%%")
        axs[0, 0].set_title("Macro Nutrients Distribution", fontsize=12, fontweight='bold', style='italic')
        
        # Bar chart for macronutrient progress
        axs[0, 1].bar([0, 1, 2], [protein_sum, fat_sum, carbs_sum], width=0.4, label='Consumed')
        axs[0, 1].bar([0.4, 1.4, 2.4], [PROTEIN_GOAL, FAT_GOAL, CARBS_GOAL], width=0.4, label='Goal')
        axs[0, 1].set_xticks([0.2, 1.2, 2.2])
        axs[0, 1].set_xticklabels(["Proteins", "Fats", "Carbs"])
        axs[0, 1].set_title("Macronutrients Progress", fontsize=12, fontweight='bold', style='italic')
        axs[0, 1].legend()

        # Pie chart for daily intake vs goal
        axs[1, 0].pie([calorie_sum, CALORIE_GOAL_LIMIT - calorie_sum], labels=["Consumed", "Remaining"], autopct="%1.1f%%")
        axs[1, 0].set_title("Daily Calorie Intake vs Goal", fontsize=12, fontweight='bold', style='italic')

        # Line plot for daily intake vs goal
        axs[1, 1].plot(["Goal", "Consumed"], [CALORIE_GOAL_LIMIT, calorie_sum], marker='o', color='blue')
        axs[1, 1].set_title("Daily Calorie Intake vs Goal", fontsize=12, fontweight='bold', style='italic')
        axs[1, 1].set_ylim(0, max(CALORIE_GOAL_LIMIT, calorie_sum) + 100)  # Set y-limit for better visualization

        plt.tight_layout()
        plt.show()
    
    elif choice == "3":
        print("Quitting...")
        done = True
    else:
        print("Invalid option. Please choose 1, 2, or 3.")